In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, re, time, math, tqdm, itertools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as pyo
# import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, re, time, math, tqdm, itertools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as pyo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
import keras
#from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, BatchNormalization, Dense
from keras.utils.np_utils import to_categorical
#from keras.models import Sequentia

In [ ]:
df_dataset = pd.read_csv('/kaggle/input/ids-intrusion-csv/02-14-2018.csv')

In [ ]:
df_dataset

In [ ]:
df_dataset.columns

In [ ]:
df_dataset["Label"].unique()

In [ ]:
# make a plot number of labels
sns.set(rc={'figure.figsize':(12, 6)})
plt.xlabel('Attack Type')
sns.set_theme()
ax = sns.countplot(x='Label', data=df_dataset)
ax.set(xlabel='Attack Type', ylabel='Number of Attacks')
plt.show()

In [ ]:
# viewing the distribution of intrusion attacks in our dataset 
plt.figure(figsize=(10, 8))
circle = plt.Circle((0, 0), 0.7, color='white')
plt.title('Intrusion Attack Type Distribution')
plt.pie(df_dataset['Label'].value_counts(), labels=['Benign', 'FTP-BruteForce','SSH-Bruteforce'], colors=['blue', 'magenta', 'cyan'])
p = plt.gcf()
p.gca().add_artist(circle)

In [ ]:
# Removing row with erroneous 'Label' label
df_dataset.drop(df_dataset.loc[df_dataset["Label"] == "Label"].index, inplace=True)

In [ ]:
df_dataset

In [ ]:
df_dataset.columns

In [ ]:
df_dataset["Label"].unique()

In [ ]:
# Remove erroneous 'Label' row
#df_dataset.drop(df_dataset.loc[df_dataset["Label"] == "Label"].index, inplace=True)

In [ ]:
df_dataset

In [ ]:
df_dataset["Label"].unique()

In [ ]:
print(df_dataset["Label"].value_counts()[['Benign']].sum())
print(df_dataset["Label"].value_counts()[['FTP-BruteForce']].sum())
print(df_dataset["Label"].value_counts()[['SSH-Bruteforce']].sum())

In [ ]:
#Checking "Protocol" values, and creating dummies based on the assumption that the Protocol values are categorical
df_dataset["Protocol"].unique()

In [ ]:
# Forcing them to be strings
df_dataset = df_dataset.astype({"Protocol": str})

In [ ]:
df_dataset["Protocol"].unique()

In [ ]:
df_dataset = pd.get_dummies(df_dataset, columns=['Protocol'], drop_first=True)

In [ ]:
df_dataset

In [ ]:
# making Label column the last column again
df_dataset.insert(len(df_dataset.columns)-1, 'Label', df_dataset.pop('Label'))

In [ ]:
#Dropping some columns that may be irrelevant (would require re-analysis)
columns_to_drop = [
    'Dst Port',
    'Timestamp',
    'Fwd PSH Flags',
    'Bwd PSH Flags',
    'Fwd URG Flags',
    'Bwd URG Flags',
    'Flow Byts/s',  # This field had np.inf values during training, as such was removed
    'Flow Pkts/s'  # This field had np.inf values during training, as such was removed
]

In [ ]:
df_dataset.drop(columns=columns_to_drop, inplace=True)
df_dataset

In [ ]:
#Dropping missing or duplicate values
df_dataset.dropna(inplace=True)
df_dataset

In [ ]:
df_dataset.drop_duplicates(inplace=True)
df_dataset

In [ ]:
print(df_dataset["Label"].value_counts()[['Benign']].sum())
print(df_dataset["Label"].value_counts()[['FTP-BruteForce']].sum())
print(df_dataset["Label"].value_counts()[['SSH-Bruteforce']].sum())

In [ ]:
#Replacing other FTP and SSH with word Malicious
df_dataset.replace(to_replace=["FTP-BruteForce","SSH-Bruteforce"], value="Malicious", inplace=True)

In [ ]:
# viewing the distribution of intrusion attacks in our dataset 
plt.figure(figsize=(10, 8))
circle = plt.Circle((0, 0), 0.7, color='white')
plt.title('Intrusion Attack Type Distribution')
plt.pie(df_dataset['Label'].value_counts(), labels=['Benign', 'Malicious'], colors=['blue', 'magenta'])
p = plt.gcf()
p.gca().add_artist(circle)

In [ ]:
#Replacing categorical values with 0 and 1
df_dataset.replace(to_replace="Benign", value=0, inplace=True)
df_dataset.replace(to_replace="Malicious", value=1, inplace=True)
df_dataset

In [ ]:
#Checking distribution of labels after dropping missing or duplicate values
print(df_dataset["Label"].value_counts()[[0]].sum())
print(df_dataset["Label"].value_counts()[[1]].sum())

In [ ]:
df_dataset.info()

In [ ]:
df_dataset

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
RANDOM_STATE_SEED = 420
#checking sample values 
df_1 = df_dataset[df_dataset.Label==0]
df_2 = df_dataset[df_dataset.Label==1]
print(len(df_1),len(df_2))

In [ ]:
#Resampling Malicious values and adjusting samples to match values of benign for avoid data imbalance
from sklearn.utils import resample
df_2_upsampled = resample(df_2, replace=True, n_samples=len(df_1))  
df_balanced = pd.concat([df_1, df_2_upsampled])
df_dataset = df_balanced
df_dataset

In [ ]:
#Training and testing split
train, test = train_test_split(df_dataset, test_size=0.3, random_state=RANDOM_STATE_SEED)
target_train = train['Label']
target_test = test['Label']
target_train.unique(), target_test.unique()

In [ ]:
test.shape

In [ ]:
y_train = to_categorical(target_train, num_classes=2)
y_test = to_categorical(target_test, num_classes=2)

In [ ]:
# making train & test splits
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = train.iloc[:, :-1].values
X_test = test.iloc[:, :-1].values
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_test

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import numpy as np

X_train = np.asarray(X_train).astype(np.float32)
#y_train = np.asarray(y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
#y_test = np.asarray(X_test).astype(np.float32)

In [ ]:
print("Full dataset:\n")
print("Benign: " + str(df_dataset["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(df_dataset["Label"].value_counts()[[1]].sum()))


print("---------------")

print("Training set:\n")
print("Benign: " + str(train["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(train["Label"].value_counts()[[1]].sum()))


print("---------------")

print("Test set:\n")
print("Benign: " + str(test["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(test["Label"].value_counts()[[1]].sum()))

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import tensorflow as tf

In [ ]:
#rbm model
class RBM(tf.keras.Model):
    def __init__(self,n_hidden,n_visible=72,learning_rate=0.01, momentum=0.95):
        super(RBM, self).__init__()
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.W = self.add_weight(name='W', shape=(n_visible, n_hidden), initializer='uniform', trainable=True)
        self.hb = self.add_weight(name='hb', shape=(n_hidden,), initializer='zeros', trainable=True)
        self.vb = self.add_weight(name='vb', shape=(n_visible,), initializer='zeros', trainable=True)

    def call(self, inputs):
        hidden_prob = tf.sigmoid(tf.matmul(inputs, self.W) + self.hb)
        hidden_state = tf.dtypes.cast(tf.random.uniform(tf.shape(hidden_prob)) < hidden_prob, tf.float32)
        visible_prob = tf.sigmoid(tf.matmul(hidden_state, tf.transpose(self.W)) + self.vb)
        return visible_prob

    def sample_h_given_v(self, v):
        return tf.sigmoid(tf.matmul(v, self.W) + self.hb)

    def sample_v_given_h(self, h):
        return tf.sigmoid(tf.matmul(h, tf.transpose(self.W)) + self.vb)

    def contrastive_divergence(self, input_data, k=1):
        v0 = input_data
        hk = self.sample_h_given_v(v0)
        for i in range(k):
            vk = self.sample_v_given_h(hk)
            hk = self.sample_h_given_v(vk)
        positive_grad = tf.matmul(tf.transpose(v0), hk)
        negative_grad = tf.matmul(tf.transpose(vk), hk)
        W_update = self.learning_rate * (positive_grad - negative_grad) / tf.dtypes.cast(tf.shape(input_data)[0], tf.float32)
        hb_update = tf.reduce_mean((hk - self.sample_h_given_v(vk)), axis=0) * self.learning_rate
        vb_update = tf.reduce_mean((v0 - vk), axis=0) * self.learning_rate
        return W_update, hb_update, vb_update


In [ ]:
#pretraining the first layer
rbm_model_1 = RBM(n_hidden=64)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
def mse_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))
dataset = tf.data.Dataset.from_tensor_slices(X_train)
epochs = 10
batch_size = 1024
for epoch in range(epochs):
    for batch in dataset.batch(batch_size):
        with tf.GradientTape() as tape:
            positive_grad, negative_grad, _ = rbm_model_1.contrastive_divergence(batch)
            reconstruction_error = mse_loss(batch, rbm_model_1(batch))
            loss = reconstruction_error
        gradients = tape.gradient(loss, rbm_model_1.trainable_variables)
        optimizer.apply_gradients(zip(gradients, rbm_model_1.trainable_variables))
    print(loss)

In [ ]:
#pretraining second layer
rbm_model_2 = RBM(n_hidden=128,n_visible=64)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
def mse_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))
dataset = tf.data.Dataset.from_tensor_slices(X_train)
epochs = 10
batch_size = 1024
for epoch in range(epochs):
    for batch in dataset.batch(batch_size):
        with tf.GradientTape() as tape:
            positive_grad, negative_grad, _ = rbm_model_2.contrastive_divergence(rbm_model_1.sample_h_given_v(batch))
            reconstruction_error = mse_loss(rbm_model_1.sample_h_given_v(batch), rbm_model_2(rbm_model_1.sample_h_given_v(batch)))
            loss = reconstruction_error
        gradients = tape.gradient(loss, rbm_model_2.trainable_variables)
        optimizer.apply_gradients(zip(gradients, rbm_model_2.trainable_variables))
    print(loss)

In [ ]:
#pretraining 3rd layer
rbm_model_3 = RBM(n_hidden=256,n_visible=128)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
def mse_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))
dataset = tf.data.Dataset.from_tensor_slices(X_train)
epochs = 10
batch_size = 1024
for epoch in range(epochs):
    for batch in dataset.batch(batch_size):
        with tf.GradientTape() as tape:
            positive_grad, negative_grad, _ = rbm_model_3.contrastive_divergence(rbm_model_2.sample_h_given_v(rbm_model_1.sample_h_given_v(batch)))
            reconstruction_error = mse_loss(rbm_model_2.sample_h_given_v(rbm_model_1.sample_h_given_v(batch)), rbm_model_3(rbm_model_2.sample_h_given_v(rbm_model_1.sample_h_given_v(batch))))
            loss = reconstruction_error
        gradients = tape.gradient(loss, rbm_model_3.trainable_variables)
        optimizer.apply_gradients(zip(gradients, rbm_model_3.trainable_variables))
    print(loss)

In [ ]:
# rbm1 = RBM(n_visible=72, n_hidden=256, learning_rate=0.001, momentum=0.95)
# rbm2 = RBM(n_visible=252, n_hidden=128, learning_rate=0.001, momentum=0.95)
# rbm3 = RBM(n_visible=128, n_hidden=64, learning_rate=0.001, momentum=0.95)

In [ ]:
# import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, re, time, math, tqdm, itertools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as pyo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
import keras
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, BatchNormalization, Dense
from keras.utils.np_utils import to_categorical
from keras.models import Sequential

In [ ]:
#Creating dbn by stacking 3 rbm layers and in last adding a classification part with dense layer and activation function
inputs = tf.keras.Input(shape=(72,))
hidden_layer1 = rbm_model_1.sample_h_given_v(inputs)
x1 = Dense(128,activation="relu")(hidden_layer1)
hidden_layer2 = rbm_model_2.sample_h_given_v(hidden_layer1)
x2 = Dense(128,activation="relu")(hidden_layer2)
hidden_layer3 =rbm_model_3.sample_h_given_v(hidden_layer2)
x3 = Dense(128,activation="relu")(hidden_layer3)
outputs = Dense(2, activation='softmax')(tf.keras.layers.Concatenate(axis=1)([x1,x2,x3]))
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
#Defining early stopping and learning coefficient
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_lr=0.0001)

In [ ]:
#Compiling model
adam = keras.optimizers.Adam(learning_rate=0.001, decay=1e-5)

model.compile( loss = "categorical_crossentropy", 
               optimizer = adam, 
               metrics=['accuracy']
             )
model.summary()

In [ ]:
#training model
history=model.fit(X_train, y_train, epochs=10, batch_size=1024, validation_data=(X_test, y_test),
        callbacks=[early_stop, reduce_lr])


In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
y_pred =np.argmax(model.predict(X_test),axis=1)

In [ ]:
y_test =np.argmax(y_test,axis=1)
y_test

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
confusion_matrix=confusion_matrix(y_test, y_pred)
confusion_matrix

In [ ]:
import seaborn as sns

In [ ]:
T5_lables = ['Benign ','Malicious']  
ax= plt.subplot()
sns.heatmap(confusion_matrix, annot=True, fmt='g', ax=ax)
# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(T5_lables); ax.yaxis.set_ticklabels(T5_lables);

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)

In [ ]:
print("F1 Score : ",f1_score(y_test, y_pred, 
                                           pos_label='positive',
                                           average='micro'))
print("Precision Score : ",precision_score(y_test, y_pred, 
                                           pos_label='positive',
                                           average='micro'))
print("Recall Score : ",recall_score(y_test, y_pred, 
                                           pos_label='positive',
                                           average='micro'))
